In [1]:
from numpy import expand_dims, zeros, ones, asarray
from numpy.random import randn, randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, \
Dropout, Lambda, Activation
import matplotlib.pyplot as plt
from keras import backend as K

In [2]:
def custom_activation(output):
    logexpsum = K.sum(K.exp(output), axis=-1, keepdims=True)
    result = logexpsum / (logexpsum + 1)
    return result

In [3]:
def define_discriminator(in_shape=(28,28,1), n_classes=10):
    in_image = Input(shape=in_shape)
    d = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    d = Conv2D(128, (3,3), strides=(2,2), padding='same')(in_image)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    d = Dense(n_classes)(d)
    c_out_layer = Activation('softmax')(d)
    c_model = Model(in_image, c_out_layer)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    c_model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    d_out_layer = Lambda(custom_activation)(d)
    d_model = Model(in_image, d_out_layer)
    d_model.compile(loss='binary_crossentropy', optimizer=opt)
    return d_model, c_model

In [4]:
def define_generator(latent_dim):
    in_lat = Input(shape=(latent_dim, ))
    g = Dense(128*7*7)(in_lat)
    g = LeakyReLU(alpha=0.2)(g)
    g = Reshape((7,7,128))(g)
    g = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(g)
    g = LeakyReLU(alpha=0.2)(g)
    g = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(g)
    g = LeakyReLU(alpha=0.2)(g)
    g_out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(g)
    model = Model(in_lat, g_out_layer)
    return model

In [5]:
def define_gan(g_model, d_model):
    d_model.trainable = False
    gan_output = d_model(g_model.output)
    model = Model(g_model.input, gan_output)
    opt = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

In [6]:
def load_real_samples():
    (trainX, trainy), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = (X - 127.5) / 127.5
    return [X, trainy]

In [7]:
def select_supervised_samples(dataset, n_samples=100, n_classes=10):
    X, y = dataset
    X_list, y_list = list(), list()
    n_per_class = int(n_samples / n_classes)
    for i in range(n_classes):
        X_with_class = X[y==i]
        idx = randint(0, len(X_with_class), n_per_class)
        [X_list.append(X_with_class[j] for j in idx)]
        [y_list.append(i) for j in idx]
    return asarray(X_list), asarray(y_list)

In [8]:
def generate_real_samples(dataset, n_samples):
    images, labels = dataset
    idx = randint(0, images.shape[0], n_samples)
    X, labels = images[idx], labels[idx]
    y = ones((n_samples, 1))
    return [X, labels], y

In [9]:
def generate_latent_points(latent_dim, n_samples):
    z_input = randn((latent_dim * n_samples))
    z_input = z_input.reshape((n_samples, latent_dim))
    return z_input

def generate_fake_samples(generator, latent_dim, n_samples):
    z_input = generate_latent_points(latent_dim, n_samples)
    images = generator.predict(z_input)
    y = zeros((n_samples, 1))
    return images, y

In [10]:
def summarize_performance(step, g_model, c_model, latent_dim, dataset, n_samples=100):
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)
    X = X + 1 / 2.0
    for i in range(100):
        plt.subplot(10,10,i+1)
        plt.axis('off')
        plt.imshow(X[i, :, :, 0], cmap='gray_r')
    filename1 = 'generated_plot_%04d.png' %(step+1)
    plt.savefig(filename1)
    plt.close()
    X, y = dataset
    _, acc = c_model.evaluate(X, y, verbose=0)
    print('classifier Accuracy: %.3f' %(acc*100))
    filename2 = 'gen_model_%04d.h5' %(step+1)
    g_model.save(filename2)
    filename3 = 'classifier_model_%04d.h5' %(step+1)
    c_model.save(filename3)
    print('>Saved: %s, %s and %s' %(filename1, filename2, filename3))

In [11]:
def train(g_model, d_model, c_model, gan_model, dataset, latent_dim, n_epochs=20, n_batch=100):
    X_sup, y_sup = select_supervised_samples(dataset)
    print(X_sup.shape, y_sup.shape)
    batch_per_epoch = int(dataset[0].shape[0] / n_batch)
    n_steps = batch_per_epoch * n_epochs
    half_batch = int(n_batch / 2)
    for i in range(n_steps):
        [Xsup_real, ysup_real], _ = generate_real_samples([X_sup, y_sup], half_batch)
        c_loss, c_acc = c_model.train_on_batch(Xsup_real, ysup_real)
        [X_real, _], y_real = generate_real_samples(dataset, half_batch)
        d_loss1 = d_model.train_on_batch(X_real, y_real)
        X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        d_loss2 = d_model.train_on_batch(X_fake, y_fake)
        X_gan = generate_latent_points(latent_dim, n_batch)
        y_gan = ones((n_batch, 1))
        gan_loss = gan_model.train_on_batch(X_gan, y_gan)
        if (i+1) % (batch_per_epoch * 1) == 0:
            summarize_performance(i, g_model,c_model, latent_dim, dataset)

In [12]:
latent_dim = 100
d_model, c_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()

In [19]:
train(g_model, d_model, c_model, gan_model, dataset, latent_dim)

In [22]:
#using the trained classifier model
model = load_model('classifier_model_9000.h5')
(trainX, trainy), (testX, testy) = load_data()
trainX = expand_dims(trainX, axis=-1)
testX = expand_dims(testX, axis=-1)
trainX = trainX.astype('float32')
testX = testX.astype('float32')
trainX = trainX - 127.5 / 127.5
testX = testX - 127.5 / 127.5
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
print('Training Accuracy: %.3f' %(train_acc*100))
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Testing Accuracy: %.3f' %(test_acc*100))